## USE pretrained network to output keypoint's description

In [2]:
from __future__ import division, print_function
import glob
import os
import cv2
import PIL
import random
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torch.nn.init
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.autograd import Variable
from copy import deepcopy, copy
from config_profile import args
from Utils import cv2_scale36, cv2_scale, np_reshape, np_reshape64
from munkres import Munkres
from descriptor_CNN3 import DesNet

Debug = False

model = DesNet()
if args.cuda:
    model.cuda()
weight_path = "checkpoint_b.pth"
trained_weight = torch.load(weight_path)
model.load_state_dict(trained_weight['state_dict'])

# load patches
patches_a_dir = "../keypoint_detector/patches-a.pt"
patches_i_dir = "../keypoint_detector/patches-i.pt"
patches_q_dir = "../keypoint_detector/patches-q.pt"
patches_a = torch.load(patches_a_dir)
patches_i = torch.load(patches_i_dir)
patches_q = torch.load(patches_q_dir)

if Debug:
    print(patches_a.shape)
    print(patches_i.shape)
    print(patches_q.shape)
patches_a =  patches_a.view(-1, 1, 32, 32).cuda()
patches_i =  patches_i.view(-1, 1, 32, 32).cuda()
patches_q =  patches_q.view(-1, 1, 32, 32).cuda()

#description = model(patches)
with torch.no_grad():
    description_a = model(patches_a)
    description_i = model(patches_i)
    description_q = model(patches_q)

description_a = description_a.view(-1, 30, 128)
description_i = description_i.view(-1, 30, 128)
description_q = description_q.view(-1, 30, 128)

torch.save(description_a, "Description.pt")

if Debug:
    print(description_a.shape)
    print(description_i.shape)
    print(description_q.shape)

print("Done!")

Done!


In [ ]:
Debug = False
similarity_matrix_one_to_one = np.zeros((35,140))
if Debug:
    print(description_q[0].shape)
    print(description_i[0].shape)
hungarian = Munkres()
for i in range(0,35):
    top = 0
    for j in range(0,140):
        dist = np.zeros((30,30))
        for k in range(0,30):
            for l in range(0,30):
                dist[k][l] = np.linalg.norm(np.subtract(description_q[i][k].cpu().numpy(), description_i[j][l].cpu().numpy()))
        cost_indecies = hungarian.compute(np.copy(dist))
        tmp = np.zeros(len(cost_indecies))
        for k in range(0,len(cost_indecies)):
            tmp[k] = np.exp(-(dist[cost_indecies[k][0]][cost_indecies[k][1]]))
        similarity_matrix_one_to_one[i][j]=tmp.sum()
        if(top < similarity_matrix_one_to_one[i][j]):
            top = similarity_matrix_one_to_one[i][j]
            if Debug:
                print((i,j))
if Debug:
    print(similarity_matrix_one_to_one)
# Check point: saving the tensor to prevent re computing it
save = False
if save:
    print("Saving a checkpoint...")
    torch.save(similarity_matrix_one_to_one, "checkpoint-1-to-1.pt")
print("Done!")


In [4]:
# start from here to reload the tensor
Debug = False
print("Loading one to one tensor from file")
similarity_matrix_one_to_one = torch.load("checkpoint-1-to-1.pt")
if Debug:
    print(similarity_matrix_one_to_one.shape)
    print(similarity_matrix_one_to_one)
print("Done!")


Loading one to one tensor from file
Done!


In [7]:
#Tensor Re-saving
similarity_matrix_one_to_one_new = torch.Tensor(similarity_matrix_one_to_one)
print(similarity_matrix_one_to_one_new.shape)
torch.save(similarity_matrix_one_to_one_new, "checkpoint-1-to-1-tensor.pt")
print("Done!")

torch.Size([35, 140])
Done!


In [ ]:
Debug = False
similarity_matrix_many_to_many = np.zeros((35,140))
threshold = 0.1
if Debug:
    print(description_q.shape)
    print(description_i.shape)
for i in range(0,35):
    top = [np.NINF,(0,0)]
    for j in range(0,140):
        dist = np.zeros(30)
        for k in range(0,30):
            tmp = np.zeros(30)
            for l in range(k,30):
                query = description_q[i][k].cpu().numpy()
                image = description_i[j][l].cpu().numpy()
                tmp[k] = np.sum(np.absolute(np.subtract(query,image)))
            dist[k] = np.linalg.norm(tmp)
        s = np.zeros(30)
        x = np.zeros(30)
        x_s = np.zeros(30)
        for k in range (0,30):
            s[k] = np.exp(-(dist[k]))
        lamda = np.sqrt(1 / np.dot(np.transpose(s), s))
        norm = np.linalg.norm(s,axis=0)
        x = np.true_divide(s, norm)
        x_s = np.copy(x)
        x_s[x_s > threshold] = 1
        x_s[x_s < threshold] = 0
        a = np.dot(np.transpose(s), x_s)
        b = lamda * (np.dot(np.transpose(x_s), x_s) - 1)
        similarity_matrix_many_to_many[i][j]= a - b
        if Debug:
            print("lamda: "+str(lamda))
            print("norm: "+str(norm))
            print("s: "+str(s))
            print("x: "+str(x))
            print("x_s: "+str(x_s))
            print("a: "+str(a))
            print("b: "+str(b))
            print("Similarity: " + str(similarity_matrix_many_to_many[i][j]))
        if (similarity_matrix_many_to_many[i][j] > top[0]):
            top = [similarity_matrix_many_to_many[i][j],(i,j)]
            print(str(top[0]) + ": " + str(top[1]))
    print(top)
    print("_____")
save = False
if save:
    print("Saving a checkpoint...")
    torch.save(similarity_matrix_many_to_many, "checkpoint-many-to-many.pt")
print("Done!")

In [8]:
# Import many to many matching here and skip last step
Debug = False
print ("Loading many to many tensor")
similarity_matrix_many_to_many = torch.load("checkpoint-many-to-many.pt")
if Debug:
    print(similarity_matrix_many_to_many.shape)
    print(similarity_matrix_many_to_many)
print("Done!")

Loading many to many tensor
Done!


In [9]:
#Tensor Re-saving
similarity_matrix_many_to_many_new = torch.Tensor(similarity_matrix_many_to_many)
print(similarity_matrix_many_to_many_new.shape)
torch.save(similarity_matrix_many_to_many_new, "checkpoint-many-to-many-tensor.pt")
print("Done!")

torch.Size([35, 140])
Done!


Recall = $\frac{TP(\hat{X})}{4}$ and Precision= $\frac{TP(\hat{X})}{ k\in {1..4}}$

In [4]:
# Load in ground truth
Debug = False
print("Loading ground truth")
y =  np.subtract(np.array([[1, 1], [1, 2], [1, 3], [1, 4], [2, 5], [2, 6], [2, 7], [2, 8], [3, 9], [3, 10], [3, 11], [3, 12], [4, 13], [4, 14], [4, 15], [4, 16], [5, 17], [5, 18], [5, 19], [5, 20], [6, 21], [6, 22], [6, 23], [6, 24], [7, 25], [7, 26], [7, 27], [7, 28], [8, 29], [8, 30], [8, 31], [8, 32], [9, 33], [9, 34], [9, 35], [9, 36], [10, 37], [10, 38], [10, 39], [10, 40], [11, 41], [11, 42], [11, 43], [11, 44], [12, 45], [12, 46], [12, 47], [12, 48], [13, 49], [13, 50], [13, 51], [13, 52], [14, 53], [14, 54], [14, 55], [14, 56], [15, 57], [15, 58], [15, 59], [15, 60], [16, 61], [16, 62], [16, 63], [16, 64], [17, 65], [17, 66], [17, 67], [17, 68], [18, 69], [18, 70], [18, 71], [18, 72], [19, 73], [19, 74], [19, 75], [19, 76], [20, 77], [20, 78], [20, 79], [20, 80], [21, 81], [21, 82], [21, 83], [21, 84], [22, 85], [22, 86], [22, 87], [22, 88], [23, 89], [23, 90], [23, 91], [23, 92], [24, 93], [24, 94], [24, 95], [24, 96], [25, 97], [25, 98], [25, 99], [25, 100], [26, 101], [26, 102], [26, 103], [26, 104], [27, 105], [27, 106], [27, 107], [27, 108], [28, 109], [28, 110], [28, 111], [28, 112], [29, 113], [29, 114], [29, 115], [29, 116], [30, 117], [30, 118], [30, 119], [30, 120], [31, 121], [31, 122], [31, 123], [31, 124], [32, 125], [32, 126], [32, 127], [32, 128], [33, 129], [33, 130], [33, 131], [33, 132], [34, 133], [34, 134], [34, 135], [34, 136], [35, 137], [35, 138], [35, 139], [35, 140]]),1)
if Debug:
    print(y)
print("Done!")

Loading ground truth
Done!


In [5]:
# Computing Precision/recall for one to one matching
Debug = False
precision_1_to_1 = np.zeros((4,similarity_matrix_one_to_one.shape[0]))
recall_1_to_1 = np.zeros((4,similarity_matrix_one_to_one.shape[0]))
for k in range(1,5):
    for i in range(0,similarity_matrix_one_to_one.shape[0]):
        y_hat = (-similarity_matrix_one_to_one[i]).argsort()[:k]
        hits = 0
        for j in range(0,k):
            index = 4 * i + j
            if y[index][1] in y_hat:
                hits += 1
        if Debug:
            y_p = [y[4 * i][1],y[4 * i + 1][1],y[4 * i + 2][1],y[4 * i + 3][1]]
            print("k: ", k)
            print("y: "+str(y_p))
            print("y_hat: " + str(y_hat))
            print("Hits: "+str(hits))
        precision_1_to_1[k-1][i] = hits/k
        recall_1_to_1[k-1][i] = hits/4
    print ("1 to 1 Precision with k = "+str(k)+": ",np.sum(precision_1_to_1[k-1])/similarity_matrix_one_to_one.shape[0])
    print("1 to 1 Recall with k = "+str(k)+":", np.sum(recall_1_to_1[k-1])/similarity_matrix_one_to_one.shape[0])
print("Done!")

1 to 1 Precision with k = 1:  0.42857142857142855
1 to 1 Recall with k = 1: 0.10714285714285714
1 to 1 Precision with k = 2:  0.5285714285714286
1 to 1 Recall with k = 2: 0.2642857142857143
1 to 1 Precision with k = 3:  0.6571428571428571
1 to 1 Recall with k = 3: 0.4928571428571429
1 to 1 Precision with k = 4:  0.75
1 to 1 Recall with k = 4: 0.75
Done!


In [6]:
# Computing Precision/recall for many to many matching
Debug = False
count = 4
precision_many_to_many = np.zeros((4,35))
recall_many_to_many = np.zeros((4,35))
for k in range(1,5):
    for i in range(0,similarity_matrix_many_to_many.shape[0]):
        y_hat = (-similarity_matrix_many_to_many[i]).argsort()[:k]
        hits = 0
        for j in range(0,k):
            index = 4 * i + j
            if y[index][1] in y_hat:
                hits += 1
        if Debug:
            y_p = [y[4 * i][1],y[4 * i + 1][1],y[4 * i + 2][1],y[4 * i + 3][1]]
            print("k: ", k)
            print("y: "+str(y_p))
            print("y_hat: " + str(y_hat))
            print("Hits: "+str(hits))
        precision_many_to_many[k-1][i] = hits/k
        recall_many_to_many[k-1][i] = hits/4
    print ("Many to Many Precision with k = "+str(k)+": ",np.sum(precision_many_to_many[k-1])/similarity_matrix_many_to_many.shape[0])
    print("Many to Many Recall with k = "+str(k)+":", np.sum(recall_many_to_many[k-1])/similarity_matrix_many_to_many.shape[0])
print("Done!")

Many to Many Precision with k = 1:  0.17142857142857143
Many to Many Recall with k = 1: 0.04285714285714286
Many to Many Precision with k = 2:  0.21428571428571427
Many to Many Recall with k = 2: 0.10714285714285714
Many to Many Precision with k = 3:  0.27619047619047615
Many to Many Recall with k = 3: 0.20714285714285716
Many to Many Precision with k = 4:  0.2714285714285714
Many to Many Recall with k = 4: 0.2714285714285714
Done!
